In [139]:
# Requests
import urllib.request #
import requests
from requests import get
from requests.exceptions import RequestException
from contextlib import closing

# for xml & html scrapping 
import lxml.html as lh
from bs4 import BeautifulSoup

# for table analysis
import pandas as pd

# write to csv
import csv

# Time
import time

import re


# Visualisation libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()






# Disable warnings 
import warnings
warnings.filterwarnings('ignore')



In [140]:
# urls
w_url = "https://en.wikipedia.org/wiki/2019%E2%80%9320_coronavirus_pandemic"
c_url = "https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Canada"
o_url = "https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Ontario"
oak_url = "https://www.halton.ca/For-Residents/Immunizations-Preventable-Disease/Diseases-Infections/New-Coronavirus"

In [141]:
# responses
w_resp = requests.get(w_url, timeout=10)
c_resp = requests.get(c_url, timeout=10)
o_resp = requests.get(o_url, timeout=10)

In [142]:
# parse response content to html
w_soup = BeautifulSoup(w_resp.content, 'html.parser')
c_soup = BeautifulSoup(c_resp.content, 'html.parser')
o_soup = BeautifulSoup(o_resp.content, 'html.parser')

In [143]:
# title
print(w_soup.title.string)
print(c_soup.title.string)
print(o_soup.title.string)

2019–20 coronavirus pandemic - Wikipedia
2020 coronavirus pandemic in Canada - Wikipedia
2020 coronavirus pandemic in Ontario - Wikipedia


In [144]:
# find data tables
w_table=w_soup.find('table', {"class":"wikitable plainrowheaders sortable"})

In [145]:
# get rows
w_rows = w_table.findAll("tr")

In [146]:
# get header attributes of the table
w_cols = [th.text.rstrip() for th in w_rows[0].find_all('th')]

In [147]:
# get data - countries
w_ctry_lst = []
for row in w_rows[2:]:
            data = [h.text.rstrip() for h in row.find_all('th')]
            if data :
                w_ctry_lst.append(re.sub(r" ?\([^)]+\)", "", data[1]))  

w_ctry_dict = {w_cols[0]:w_ctry_lst}
w_ctry_df = pd.DataFrame(w_ctry_dict)
print(w_ctry_df.head(3))
print('----------------')
print(w_ctry_df.count())

       Locations[b]
0  United States[e]
1          Italy[f]
2          China[g]
----------------
Locations[b]    183
dtype: int64


In [148]:
# get data - Covid Stats
w_data_lst = []
for row in w_rows[2:]:
            data = [d.text.rstrip() for d in row.find_all('td')]
            w_data_lst.append(data)

w_df_lst = []
for d in w_data_lst:
    lst_df = pd.DataFrame(d)
    w_df_lst.append(lst_df.T)

w_data_df = pd.concat(w_df_lst,axis=0)
w_data_df = w_data_df.dropna(how='any')
w_data_df.head(3)   

0       1       2         3
0  119,938   1,991   3,229  [38][39]
0   92,472  10,023  12,384      [42]
0   81,394   3,295  74,971      [43]

In [149]:
w_ctry_df.reset_index(drop=True, inplace=True)
w_data_df.reset_index(drop=True, inplace=True)
w_df = pd.concat([w_ctry_df, w_data_df], axis=1)

In [150]:
w_df.columns = w_cols

In [151]:
# drop 'comm'column 
w_df = w_df.drop(w_df[['Ref.']],axis=1)
w_df = w_df.replace(',','', regex=True)
w_cols1 = ["Country", "Confirmed","Deaths","Recovered" ]
w_df.columns = w_cols1
w_df.head(5)

Country Confirmed Deaths Recovered
0  United States[e]    119938   1991      3229
1          Italy[f]     92472  10023     12384
2          China[g]     81394   3295     74971
3          Spain[h]     72251   5812     12285
4           Germany     56493    421      4107

In [152]:
# convert just columns "a" and "b"
w_df[w_cols1[1:]] = w_df[w_cols1[1:]].apply(pd.to_numeric,errors='coerce').fillna(0).astype(int)

In [153]:
w_df.dtypes

Country      object
Confirmed     int32
Deaths        int32
Recovered     int32
dtype: object

In [154]:
w_df

Country  Confirmed  Deaths  Recovered
0                  United States[e]     119938    1991       3229
1                          Italy[f]      92472   10023      12384
2                          China[g]      81394    3295      74971
3                          Spain[h]      72251    5812      12285
4                           Germany      56493     421       4107
5                         France[i]      37611    2314       5700
6                           Iran[j]      35408    2517      11679
7                 United Kingdom[k]      17089    1028        140
8                       Switzerland      14108     271       1595
9                    Netherlands[l]       9806     640          0
10                      South Korea       9478     144       4811
11                          Belgium       9134     353       1063
12                          Austria       8233      68        225
13                           Turkey       7402     108         70
14                           Canada       5362      59        353
15                         Portugal       5170     100         43
16                        Norway[m]       3988      23          0
17                     Australia[n]       3636      14        224
18                           Brazil       3477      93          6
19                           Israel       3460      12         89
20                        Sweden[o]       3460     110         18
21                   Czech Republic       2541       9         11
22                       Denmark[p]       2366      65          0
23                         Malaysia       2320      27        320
24                          Ireland       2121      22          5
25                            Chile       1909       6         61
26                       Luxembourg       1831      18         40
27                          Ecuador       1823      48          3
28                           Poland       1638      20         21
29                            Japan       1499      49        359
..                              ...        ...     ...        ...
153                         Grenada          7       0          0
154                        Zimbabwe          7       1          0
155                           Benin          6       0          0
156                         Eritrea          6       0          0
157                            Fiji          5       0          0
158                           Sudan          5       1          0
159                      Mauritania          5       0          0
160                           Nepal          5       0          1
161                           Syria          5       0          0
162                          Angola          4       0          0
163                      Cape Verde          4       1          0
164           Republic of the Congo          4       0          0
165                       Nicaragua          4       1          0
166                    Vatican City          4       0          0
167             Antigua and Barbuda          3       0          0
168                          Bhutan          3       0          0
169        Central African Republic          3       0          0
170                            Chad          3       0          0
171                          Gambia          3       1          0
172                         Liberia          3       0          0
173                           Libya          3       0          0
174                         Somalia          3       0          0
175                     Saint Lucia          3       0          0
176                          Belize          2       0          0
177                   Guinea-Bissau          2       0          0
178                   MS Zaandam[z]          2       0          0
179           Saint Kitts and Nevis          2       0          0
180                      East Timor          1       0          0
181                Papua New Guinea          1       0          0
18

In [155]:
# find data tables
c_table=c_soup.find_all('table', {"class":"wikitable sortable"})
c_table1 = c_table[1]

In [156]:
# get rows
c_rows = c_table1.findAll("tr")

In [157]:
# get header attributes of the table
c_cols = [th.text.rstrip() for th in c_rows[1].find_all('th')]
print(c_cols)

['Province', 'Tests', 'Conf.', 'Pres.', 'Total', 'Population', 'Per m', 'Recov.', 'Deaths', 'Active', 'Ref.']


In [158]:
# get data - Covid Stats
c_data_lst = []
for row in c_rows[3:]:
            data = [d.text.rstrip() for d in row.find_all('td')]
            c_data_lst.append(data)

In [159]:
c_df_lst = []
for d in c_data_lst:
    lst_df = pd.DataFrame(d)
    c_df_lst.append(lst_df.T)

c_data_df = pd.concat(c_df_lst,axis=0)
c_data_df = c_data_df.dropna(how='any')

In [160]:
c_data_df.columns = c_cols

# drop 'comm'column 
c_data_df = c_data_df.drop(c_data_df[['Ref.']],axis=1)
c_data_df

Province   Tests  Conf. Pres.  Total  Population  Per m  \
0                     Alberta  38,215    542     0    542   4,413,146  110.1   
0                Saskatchewan   6,915     95     0     95   1,181,666   80.4   
0                    Manitoba   6,205     25    14     39   1,377,517   28.3   
0                     Ontario  41,032    993     0    993  14,711,827   67.5   
0                      Quebec  43,589  2,498     0  2,498   8,537,674  236.7   
0               New Brunswick   1,828     45     0     45     779,993   57.7   
0        Prince Edward Island     556     11     0     11     158,158   69.6   
0                 Nova Scotia   3,739     90     0     90     977,457   92.1   
0   Newfoundland and Labrador   1,491    120     0    102     521,365  195.6   
0                       Yukon     517      3     0      3      41,078   73.0   
0       Northwest Territories     559      1     0      1      44,904   22.3   
0                     Nunavut     108      0     0      0      39,097    0.0   
0      Repatriated travellers             13     0     13         N/A    N/A   

  Recov. Deaths Active  
0     33      2    457  
0      3      0     92  
0      0      1     38  
0      8     18    967  
0     29     22  2,477  
0      0      0     45  
0      1      0     10  
0      2      0     88  
0      0      0    102  
0      0      0      3  
0      0      0      1  
0      0      0      0  
0      0      0     13

In [47]:
# find data tables
c_table2=c_soup.find('table',{"class":"wikitable"})

In [168]:
# find data tables
o_table=o_soup.find('table', {"class":"wikitable sortable"})

In [169]:
# get rows
o_rows = o_table.findAll("tr")

In [170]:
# get header attributes of the table
o_cols = [th.text.rstrip() for th in o_rows[0].find_all('th')]
print(o_cols)

['Public Health Unit', 'Cases', 'Per m', 'Deaths', 'Ref']


In [171]:
# get data - Covid Stats
o_data_lst = []
for row in o_rows[1:]:
            data = [d.text.rstrip() for d in row.find_all('td')]
            o_data_lst.append(data)

In [172]:
o_df_lst = []
for d in o_data_lst:
    lst_df = pd.DataFrame(d)
    o_df_lst.append(lst_df.T)

o_data_df = pd.concat(o_df_lst,axis=0)
o_data_df = o_data_df.dropna(how='any')

In [173]:
o_data_df.columns = o_cols
# drop 'comm'column 
o_data_df = o_data_df.drop(o_data_df[['Ref']],axis=1)
o_data_df

Public Health Unit Cases   Per m Deaths
0                                             Algoma     1    8.85       
0                 Brant County (including Brantford)     7   51.85       
0                                       Chatham-Kent     4   39.22      0
0                                      Durham Region    86  132.31      3
0  Eastern Ontario (Stormont, Dundas and Glengarr...     4   19.70      0
0                                         Grey Bruce     7   43.21       
0                                  Haldimand-Norfolk     5   45.45      1
0          Haliburton, Kawartha, Pine Ridge District    33  184.36      3
0                                      Halton Region    20   36.36      1
0                                           Hamilton    51   94.44      1
0                             Hastings Prince Edward     6   37.27       
0                                        Huron-Perth     5   36.76       
0         Kingston, Frontenac and Lennox & Addington    17   89.47       
0                                     Lambton County    10   81.30       
0                 Leeds, Grenville & Lanark District     2   15.40       
0                                   Middlesex-London    25   54.35      0
0                                     Niagara Region    12   26.67       
0  North Bay-Parry Sound (including most of Nipis...     1    8.13       
0  Northwestern (Most of Kenora District, Rainy R...     2   26.32       
0                            Ottawa (confirmed only)    75   80.65      1
0                                               Peel   154  111.59       
0                     Peterborough (City and County)    13   94.20      0
0  Porcupine (Cochrane District, Hornepayne, Jame...    10  119.05       
0                          Renfrew County & District     1    9.62       
0                                     Simcoe-Muskoka    33   61.11      3
0           Southwestern (Oxford, Elgin, St. Thomas)     5   25.13      0
0    Sudbury & Districts (including Greater Sudbury)     6   30.61       
0  Thunder Bay District (including First Nations ...     1    6.58       
0                   Timiskaming (including Temagami)     1   30.30       
0                                            Toronto   457  167.40      4
0                                  Waterloo (Region)    69  127.78      0
0                         Wellington-Dufferin-Guelph     7   25.00      0
0                               Windsor-Essex County    15   37.59       
0                                        York Region   163  146.85      2

In [174]:
o_data_df.dtypes

Public Health Unit    object
Cases                 object
Per m                 object
Deaths                object
dtype: object

In [175]:
o_cols = list(o_data_df.columns)
o_cols[1:]

['Cases', 'Per m', 'Deaths']

In [179]:
# convert just columns "a" and "b"
o_data_df[o_cols[1:]] = o_data_df[o_cols[1:]].apply(pd.to_numeric).fillna(0)
o_data_df.dtypes

Public Health Unit     object
Cases                   int64
Per m                 float64
Deaths                float64
dtype: object

In [180]:
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: pink' if v else '' for v in is_max]

o_data_df.reset_index(drop=True).style.apply(highlight_max,subset=['Per m'])

In [ ]:
# find all the tables in the html
all_tables=c_soup.find_all('table')

In [ ]:
rows = right_table.findAll("tr")
len(rows)

In [ ]:
# header attributes of the table
header = [th.text.rstrip() for th in rows[0].find_all('th')]
print(header[0])
print('------------')
print(len(header))

In [ ]:
# header attributes of the table
cols1 = [th.text.rstrip() for th in rows[2].find_all('th')]
print(cols1)
print('------------')
print(len(cols1))